# S&DS 617 Applied Machine Learning and Causal Inference Research Seminar: Assignment 1

**Deadline**

Assignment 1 is due Monday, February 24th at 1:30pm. Late work will not be accepted. 

**Submission**

Submit your assignment as a .pdf on Gradescope. On Gradescope, there are 2 assignments, one where you will submit a pdf file and one where you will submit the corresponding .ipynb that generated it. 
Note: The problems in each homework assignment are numbered. When submitting the pdf on Gradescope, please select the correct pages that correspond to each problem. 

To produce the .pdf, do the following to preserve the cell structure of the notebook:
- Go to "File" at the top-left of your Jupyter Notebook
- Under "Download as", select "HTML (.html)"
- After the .html has downloaded, open it and then select "File" and "Print"
- From the print window, select the option to save as a .pdf

## Problem 1: Comparing BERT vs. GPT

a) In this assignment, we will compare BERT (Bidirectional Encoder Representations from Transformers) with GPT (Generative Pre-training Transformer). Provide detailed explanations of how the architecture, the type of attention mechanism employed, and the approach to tokenization in each model contribute to their respective capabilities and applications. Which model do you think will perform better at sentiment analysis and why?

b) We will now perform sentiment analysis on the IMDb dataset ("https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"). This dataset contains movie reviews along with their associated binary sentiment polarity labels. Code has been provided to you below to train and evaluate BERT. 

Run the below code to get the test accuracy. Then, modify the code to try getting a higher test accuracy (e.g., adjusting hyperparameters, further model tweaking, data augmentation, etc.). Specify what you modified.

In [11]:
import requests
import tarfile
import os
import json
import re
import openai
from io import BytesIO
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from torch.utils.data import Dataset

### Get Data

In [ ]:
# URL of the IMDb dataset
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

# Send a GET request to download the content of the dataset
response = requests.get(url)
response.raise_for_status()  # This will raise an exception if there was a download issue

# Open the downloaded content as a file-like object
file_like_object = BytesIO(response.content)

# Extract the tar.gz file
with tarfile.open(fileobj=file_like_object) as tar:
    tar.extractall(path=".")  # Extract to a directory named aclImdb in the current working directory

print("Dataset downloaded and extracted to './aclImdb")


In [ ]:
def load_imdb_dataset(directory):
    reviews = []
    sentiments = []

    for sentiment in ["pos", "neg"]:
        dir_name = os.path.join(directory, sentiment)
        for filename in os.listdir(dir_name):
            if filename.endswith('.txt'):
                with open(os.path.join(dir_name, filename), encoding='utf-8') as file:
                    reviews.append(file.read())
                    sentiments.append(sentiment)

    return pd.DataFrame({'review': reviews, 'sentiment': sentiments})

# Load the training dataset
dataset_dir = 'aclImdb'
df_tr = load_imdb_dataset(os.path.join(dataset_dir, 'train'))

# Load the test dataset
df_te = load_imdb_dataset(os.path.join(dataset_dir, 'test'))

# Display the first few rows of the DataFrame
print(df_tr.head())
print(df_te.head())


In [ ]:
# Subsample train and test sets down (note: you may change the size of training) 
df_tr = df_tr.sample(n=1000, random_state=928)
print(df_tr.shape) # check dimensions
df_te = df_te.sample(n=500, random_state=2755)
print(df_te.shape) # check dimensions
df_te.iloc[1, 0] # sample movie review

In [15]:
class IMDbDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)
    

### Train BERT (Note: this may take a considerable amount of time. You may modify the size of training if too computationally intensive)

In [ ]:
from transformers import TrainingArguments, Trainer, BertTokenizer, BertForSequenceClassification

# Function to tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# Load the dataset (assuming df_tr is your loaded DataFrame)
texts = df_tr['review'].tolist()
labels = df_tr['sentiment'].apply(lambda x: 1 if x == 'pos' else 0).tolist()

# Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the data
ml = 128
tokenized_dataset = tokenizer(texts, padding=True, truncation=True, max_length=ml)

# Splitting the dataset into training and validation sets
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_masks, val_masks, train_labels, val_labels = train_test_split(
    tokenized_dataset['input_ids'], tokenized_dataset['attention_mask'], labels, test_size=0.2
)

# Creating dataset objects for training and validation
class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = IMDbDataset({'input_ids': train_texts, 'attention_mask': train_masks}, train_labels)
val_dataset = IMDbDataset({'input_ids': val_texts, 'attention_mask': val_masks}, val_labels)

# Load BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Define the training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch",  # Ensure models are saved at each epoch
    evaluation_strategy="epoch",  # Evaluate at each epoch
    optim="adamw_torch",  # Use the recommended optimizer
)


# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Train the model
trainer.train()


In [ ]:
# Evaluate the model on the validation set
predictions = trainer.predict(val_dataset)
val_accuracy = accuracy_score(val_labels, predictions.predictions.argmax(-1))
print(f"Validation Accuracy: {val_accuracy}")

### Evaluate model on test set

In [ ]:
test_texts = df_te['review'].tolist()
test_labels = df_te['sentiment'].apply(lambda x: 1 if x == 'pos' else 0).tolist()

# Tokenize the test data
test_encodings = tokenizer(test_texts, padding=True, truncation=True, max_length=ml)

class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = {key: torch.tensor(val) for key, val in encodings.items()}
        self.labels = torch.tensor(labels)

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]  
        return item

    def __len__(self):
        return len(self.labels)

test_dataset = IMDbDataset(test_encodings, test_labels)

# Predictions
test_predictions = trainer.predict(test_dataset)
test_accuracy = accuracy_score(test_labels, test_predictions.predictions.argmax(-1))
print(f"Test Accuracy: {test_accuracy}")

c) Perform sentiment analysis using GPT-3.5-turbo, gpt-4o, o1-mini, and o3-mini and get the test accuracy. Evaluate their performance by comparing test accuracies. (If you get a rate limit error, just use 4o)

**Note: DO NOT try to run advanced models on the entire test set initially.** Be mindful of API usage limits and costs associated with the advanced models APIs. Start with a smaller subset of your test set to ensure your implementation is correct before scaling up. 

d) For the task of language translation, do you expect BERT or GPT to perform better? Explain why in detail. Additionally, discuss the primary challenges associated with implementing each model for translation tasks.